In [1]:
%load_ext autoreload
%autoreload 2

### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [2]:
import numpy as np

#### generate a simple linear dataset y=xb+e
##### store validation data from same dataset too

In [3]:
import data_gen as dg
data1=dg.data_gen(data_shape=(500,2),seed=0)
#xdata=data1.x[:,:-1]#drop last x
size=60
xdata=data1.x[0:size,:]
ydata=data1.y[0:size]
y=ydata
xfull=xdata
x=xdata[:,1:] #drop first column of x (all 1's)
e=data1.e
n,p=x.shape


xvalidate=data1.x[100:500,1:]
yvalidate=data1.y[100:500]

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across the nout axis (the number of possibile values y is averaged over, whose probabilities sum to 1)
  - 'own_n' means same as self, but divided by count of non-masked items in second to last, nout lenght dimension
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [4]:
max_bw_Ndiff=2
modeldict1={
    'Ndiff_type':'product',
    'max_bw_Ndiff':max_bw_Ndiff,
    'normalize_Ndiffwtsum':'own_n',
    'xkern_grid':'no',
    'ykern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'self',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'x_bandscale':'non-neg',
        'Ndiff_depth_bw':'non-neg',
        'outer_x_bw':'non-neg',
        'outer_y_bw':'non-neg',
        'y_bandscale':'fixed'
        }
    }

#modeldict1['ykern_grid']=20#override
modeldict1['product_kern_norm']='none'

#set starting or fixed values for hyper parameters
onethenzeros=np.zeros([max_bw_Ndiff,])+.0005
#if max_bw_Ndiff>0:onethenzeros[0]=1
hyper_paramdict1={
    'Ndiff_exponent':onethenzeros,
    'x_bandscale':np.ones([p,]),
    'outer_x_bw':np.array([0.3,]),
    'outer_y_bw':np.array([0.3,]),
    'Ndiff_depth_bw':np.ones([max_bw_Ndiff,])*0.3,
    'y_bandscale':np.ones([1,])
    }


#assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
#                                                                            'depth of max_bw_Ndiff'
assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
                                                                             'depth of max_bw_Ndiff'
#create optimization dictionary
optimizedict1={
    'method':'Nelder-Mead',
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

#-----------------Calibrate/Optimize--------

In [5]:
import mykern as mk



In [6]:
import myreg
just_linreg=myreg.reg(y,xfull)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

linreg_mse: 1.4299524383095898


In [ ]:
#y=np.linspace(0,10000,n)#what happens if y has no relationship to x
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)
print(optimized_Ndiff_kernel.mse)

In [ ]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

In [ ]:
#xgrid=np.linspace(1,n,n)

xgrid=x[:,0]
yhat=optimized_Ndiff_kernel.yhat_un_std
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=10,color='blue',alpha=0.4,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',alpha=0.9,legend='yhat')
p.circle(xgrid,just_linreg.yhat,color='red',legend='just lin reg yhat')
p.legend.location = "top_left"
p.yaxis.visible=False
show(p)

In [ ]:
yhat.shape
print(data1.b)

In [ ]:
print(yhat)

In [ ]:
print(y)

In [ ]:
ypredict=optimized_Ndiff_kernel.predict_tool(xvalidate,optimized_Ndiff_kernel.mselist[-1][1],modeldict1)
predition_err=yvalidate-ypredict
prediction_mse=np.mean(prediction_err**2)

In [ ]:
np.ma.count_masked(yhat)